In [1]:
import numpy as np
import plotly.graph_objects as go 
import pyrender 
import open3d as o3d
import trimesh

## Superquadrics

A thorough overview of superquadrics: https://cse.buffalo.edu/~jryde/cse673/files/superquadrics.pdf

### Implicit Equation
The surface of a basic superquadric is given by: 
$$ \vert x \vert^r + \vert y \vert^s + \vert z \vert^t = 1 ,$$

where $r, s$ and $t$ are positive real numbers that determine the main features of the superquadric:
 - $< 1$ : pointy octahedron
 - $= 1$: regular octahedron
 - $range(1, 2)$: blunt octahedron
 - $=2$ : sphere
 - $>2$ : cube modified to have rounded edges and corners
 
 We can scale these basic shapes as follows: 
 
 $$ \vert \frac{x}{A} \vert^r + \vert \frac{y}{B} \vert^s + \vert \frac{z}{C} \vert^t = 1 $$


### Parametric description

We can describe a superquadric with a parametric equation in terms of surface parameters u (longitude) and v (latitude) by:

\begin{align}
& x(u,v) = Ag\left(v, \frac{2}{r}\right)g\left(u, \frac{2}{r}\right) \\
& y(u,v) = Bg\left(v, \frac{2}{s}\right)f\left(u, \frac{2}{s}\right) \\
& z(u,v) = Cf\left(v, \frac{2}{t}\right) \\
& -\frac{\pi}{2} \leq v \leq \frac{\pi}{2}, \quad -\pi \leq u \leq \pi
\end{align}

The auxilary functions $f$ and $g$ are:

\begin{align}
& f(\omega, m) = sgn(\sin \omega) \vert \sin \omega \vert^m \\
& f(\omega, m) = sgn(\cos \omega) \vert \cos \omega \vert^m \\
\end{align}

where $sgn(x)$ is the sign function (-1 if x < 0, 0 if x == 0, 1 if x > 0) 

In [49]:
def fexp(omega, m):
    return np.sign(omega) * np.power(np.abs(omega), m)


def get_eta_and_w(n, eta_max=np.pi/2, w_max=np.pi):
    '''
    Taken from: https://github.com/HawaiiZeng/Superquadrics/blob/master/superquadrics.py
    '''
    eta_min = -eta_max
    w_min = -w_max
    d_w = (w_max - w_min) / n
    
    # When the range = 2PI, one point will be overlapped
    x = np.linspace(0, n, n + 1)
    if eta_max == np.pi / 2:
        n = n - 1
    y = np.linspace(0, n, n + 1)
    
    d_eta = (eta_max - eta_min) / n

    yv, xv = np.meshgrid(y, x)
    eta = eta_min + yv * d_eta
    w = w_min + xv * d_w
    return eta, w

from numpy import sign as sgn

def signed_sin(w, m):
    return sgn(np.sin(w)) * np.power(np.abs(np.sin(w)), m)

def signed_cos(w, m):
    return sgn(np.cos(w)) * np.power(np.abs(np.cos(w)), m)

def signed_tan(w, m):
    return sgn(np.tan(w)) * np.power(np.abs(np.tan(w)), m)

def signed_sec(w, m):
    return sgn(np.cos(w)) * np.power(np.abs(1 / np.cos(w)), m)


def superhyperboloid(epsilon, a, n=50):
    """
    https://cse.buffalo.edu/~jryde/cse673/files/superquadrics.pdf EQ 2.20
    """
    eta, w = get_eta_and_w(n)
    x = a[0] * signed_sec(eta, epsilon[0]) * signed_cos(w, epsilon[0])
    y = a[1] * signed_sec(eta, epsilon[1]) * signed_sin(w, epsilon[1])
    z = a[2] * signed_tan(eta, epsilon[2])
    return x, y, z
    
def supertoroid(epsilon, A, n=50):
    '''
        ref: https://cse.buffalo.edu/~jryde/cse673/files/superquadrics.pdf EQ 2.22
        A is now a list of 4 scaling factors
    '''
    eta, w = get_eta_and_w(n, eta_max=np.pi, w_max=np.pi)
    
    x = A[0] * (A[3] + fexp(np.cos(eta), epsilon[0])) * fexp(np.cos(w), epsilon[0])
    y = A[1] * (A[3] + fexp(np.cos(eta), epsilon[1])) * fexp(np.sin(w), epsilon[1])
    z = A[3] * fexp(np.sin(eta), epsilon[2])
    
    return x,y,z
    
def superquadric(epsilon, A, n=50):
    '''
    This follows the equation from: https://en.wikipedia.org/wiki/Superquadrics
    epsilon is a 3-element vector: 2/r, 2/s, 2/t
    A is a 3-element vector: A, B, C
    '''
    eta, w = get_eta_and_w(n)
    
    x = A[0] * fexp(np.sin(eta), epsilon[0]) * fexp(np.sin(w), epsilon[0])
    y = A[1] * fexp(np.sin(eta), epsilon[1]) * fexp(np.cos(w), epsilon[1])
    z = A[2] * fexp(np.cos(eta), epsilon[2])
    
    return x,y,z

In [50]:
def save_to_obj(save_path, x, y, z):
    """
    This function saves super-ellpsoid w/o overlap: the rightmost vertices are not coincide with the leftmost points,
    and only one vertex at the top and bottom
    """
    x = np.transpose(x, (1, 0))
    y = np.transpose(y, (1, 0))
    z = np.transpose(z, (1, 0))
    print(x.shape, y.shape, z.shape)
    hei, wid = x.shape[0], x.shape[1]
    count = 0
    with open(save_path, "w+") as fout:
        # vertices of body
        for i in range(1, hei - 1):
            for j in range(0, wid - 1):
                fout.write("v %.3f %.3f %.3f\n" % (x[i, j], y[i, j], z[i, j]))
                count += 1
        # faces
        for i in range(0, hei - 3):
            for j in range(0, wid - 2):
                fout.write("f %d %d %d %d\n" % (i * (wid - 1) + j + 1, i * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 1))
        for i in range(0, hei - 3):
            fout.write("f %d %d %d %d\n" % (i * (wid - 1) + wid - 2 + 1, i * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + wid - 2 + 1))
        fout.write("v %.3f %.3f %.3f\n" % (x[0, 0], y[0, 0], z[0, 0]))
        fout.write("v %.3f %.3f %.3f\n" % (x[-1, -1], y[-1, -1], z[-1, -1]))
        for j in range(0, wid - 2):
            fout.write("f %d %d %d\n" % (count+1, j + 2, j + 1))
        fout.write("f %d %d %d\n" % (count+1, 1, wid - 2 + 1))
        for j in range(0, wid - 2):
            fout.write("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + j + 1, (hei - 3) * (wid - 1) + j + 2))
        fout.write("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + wid - 3 + 2, (hei - 3) * (wid - 1) + 1))


In [92]:
epsilon = [2, 2, 2]
scaling = [1,1,1]

x,y,z = superquadric(epsilon, scaling)
save_to_obj('objects/superquadric.obj', x, y, z)


x,y,z = superhyperboloid(epsilon, scaling)
save_to_obj('objects/superhyperboloid.obj', x, y, z)


x,y,z = supertoroid(epsilon, [1,1,1,1], n=200)
save_to_obj('objects/supertoroid.obj', x, y, z)


(50, 51) (50, 51) (50, 51)
(50, 51) (50, 51) (50, 51)
(201, 201) (201, 201) (201, 201)


In [93]:
s_toroid = trimesh.load('objects/supertoroid.obj')
s_hyperboloid = trimesh.load('objects/superhyperboloid.obj')
s_quadric = trimesh.load('objects/superquadric.obj')
toroid_mesh = pyrender.Mesh.from_trimesh(s_toroid)
hyperboloid_mesh = pyrender.Mesh.from_trimesh(s_hyperboloid)
quadric_mesh = pyrender.Mesh.from_trimesh(s_quadric)

scene = pyrender.Scene()
scene.add(toroid_mesh)
# scene.add(hyperboloid_mesh)
# scene.add(quadric_mesh)
pyrender.Viewer(scene, use_raymond_lighting=True)

WARNING - 2020-11-03 16:00:17,893 - obj - faces have mixed data, using slow fallback!
WARNING - 2020-11-03 16:00:18,219 - obj - faces have mixed data, using slow fallback!
WARNING - 2020-11-03 16:00:18,244 - obj - faces have mixed data, using slow fallback!


Viewer(width=640, height=480)

In [14]:
from skimage import measure
from skimage.draw import ellipsoid

In [15]:
ellip_base = ellipsoid(6, 10, 16, levelset=True)
ellip_double = np.concatenate((ellip_base[:-1, ...],
                               ellip_base[2:, ...]), axis=0)
ellip_base.shape, ellip_double.shape

((15, 23, 35), (27, 23, 35))

In [31]:
x = np.transpose(x, (1, 0))
y = np.transpose(y, (1, 0))
z = np.transpose(z, (1, 0))
print(x.shape, y.shape, z.shape)
hei, wid = x.shape[0], x.shape[1]
count = 0

for i in range(1, hei - 1):
    for j in range(0, wid - 1):
        print((i,j), "v %.3f %.3f %.3f\n" % (x[i, j], y[i, j], z[i, j]))
        count += 1
for i in range(0, hei - 3):
    for j in range(0, wid - 2):
        print((i,j), "f %d %d %d %d\n" % (i * (wid - 1) + j + 1, i * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 1))
for i in range(0, hei - 3):
    print("f %d %d %d %d\n" % (i * (wid - 1) + wid - 2 + 1, i * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + wid - 2 + 1))
print("v %.3f %.3f %.3f\n" % (x[0, 0], y[0, 0], z[0, 0]))
print("v %.3f %.3f %.3f\n" % (x[-1, -1], y[-1, -1], z[-1, -1]))
for j in range(0, wid - 2):
    print("f %d %d %d\n" % (count+1, j + 2, j + 1))
print("f %d %d %d\n" % (count+1, 1, wid - 2 + 1))
for j in range(0, wid - 2):
    print("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + j + 1, (hei - 3) * (wid - 1) + j + 2))
print("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + wid - 3 + 2, (hei - 3) * (wid - 1) + 1))


(50, 51) (50, 51) (50, 51)
(1, 0) v 0.004 0.000 0.996

(1, 1) v 0.004 0.000 0.996

(1, 2) v 0.004 0.000 0.996

(1, 3) v 0.004 0.001 0.996

(1, 4) v 0.003 0.001 0.996

(1, 5) v 0.003 0.001 0.996

(1, 6) v 0.002 0.002 0.996

(1, 7) v 0.002 0.002 0.996

(1, 8) v 0.001 0.003 0.996

(1, 9) v 0.001 0.003 0.996

(1, 10) v 0.000 0.004 0.996

(1, 11) v 0.000 0.004 0.996

(1, 12) v 0.000 0.004 0.996

(1, 13) v 0.000 0.004 0.996

(1, 14) v 0.000 0.004 0.996

(1, 15) v 0.000 0.004 0.996

(1, 16) v 0.001 0.003 0.996

(1, 17) v 0.001 0.003 0.996

(1, 18) v 0.002 0.002 0.996

(1, 19) v 0.002 0.002 0.996

(1, 20) v 0.003 0.001 0.996

(1, 21) v 0.003 0.001 0.996

(1, 22) v 0.004 0.001 0.996

(1, 23) v 0.004 0.000 0.996

(1, 24) v 0.004 0.000 0.996

(1, 25) v 0.004 0.000 0.996

(1, 26) v 0.004 0.000 0.996

(1, 27) v 0.004 0.000 0.996

(1, 28) v 0.004 0.001 0.996

(1, 29) v 0.003 0.001 0.996

(1, 30) v 0.003 0.001 0.996

(1, 31) v 0.002 0.002 0.996

(1, 32) v 0.002 0.002 0.996

(1, 33) v 0.001 0.003 0.99

(19, 4) v 0.676 0.204 0.119

(19, 5) v 0.576 0.304 0.119

(19, 6) v 0.468 0.413 0.119

(19, 7) v 0.358 0.523 0.119

(19, 8) v 0.253 0.628 0.119

(19, 9) v 0.160 0.721 0.119

(19, 10) v 0.084 0.797 0.119

(19, 11) v 0.031 0.850 0.119

(19, 12) v 0.003 0.877 0.119

(19, 13) v 0.003 0.877 0.119

(19, 14) v 0.031 0.850 0.119

(19, 15) v 0.084 0.797 0.119

(19, 16) v 0.160 0.721 0.119

(19, 17) v 0.253 0.628 0.119

(19, 18) v 0.358 0.523 0.119

(19, 19) v 0.468 0.413 0.119

(19, 20) v 0.576 0.304 0.119

(19, 21) v 0.676 0.204 0.119

(19, 22) v 0.761 0.119 0.119

(19, 23) v 0.826 0.054 0.119

(19, 24) v 0.867 0.014 0.119

(19, 25) v 0.881 0.000 0.119

(19, 26) v 0.867 0.014 0.119

(19, 27) v 0.826 0.054 0.119

(19, 28) v 0.761 0.119 0.119

(19, 29) v 0.676 0.204 0.119

(19, 30) v 0.576 0.304 0.119

(19, 31) v 0.468 0.413 0.119

(19, 32) v 0.358 0.523 0.119

(19, 33) v 0.253 0.628 0.119

(19, 34) v 0.160 0.721 0.119

(19, 35) v 0.084 0.797 0.119

(19, 36) v 0.031 0.850 0.119

(19, 37) v 0.003

(34, 4) v 0.517 0.156 0.327

(34, 5) v 0.440 0.232 0.327

(34, 6) v 0.357 0.315 0.327

(34, 7) v 0.273 0.399 0.327

(34, 8) v 0.193 0.480 0.327

(34, 9) v 0.122 0.551 0.327

(34, 10) v 0.064 0.608 0.327

(34, 11) v 0.024 0.649 0.327

(34, 12) v 0.003 0.670 0.327

(34, 13) v 0.003 0.670 0.327

(34, 14) v 0.024 0.649 0.327

(34, 15) v 0.064 0.608 0.327

(34, 16) v 0.122 0.551 0.327

(34, 17) v 0.193 0.480 0.327

(34, 18) v 0.273 0.399 0.327

(34, 19) v 0.357 0.315 0.327

(34, 20) v 0.440 0.232 0.327

(34, 21) v 0.517 0.156 0.327

(34, 22) v 0.582 0.091 0.327

(34, 23) v 0.631 0.042 0.327

(34, 24) v 0.662 0.011 0.327

(34, 25) v 0.673 0.000 0.327

(34, 26) v 0.662 0.011 0.327

(34, 27) v 0.631 0.042 0.327

(34, 28) v 0.582 0.091 0.327

(34, 29) v 0.517 0.156 0.327

(34, 30) v 0.440 0.232 0.327

(34, 31) v 0.357 0.315 0.327

(34, 32) v 0.273 0.399 0.327

(34, 33) v 0.193 0.480 0.327

(34, 34) v 0.122 0.551 0.327

(34, 35) v 0.064 0.608 0.327

(34, 36) v 0.024 0.649 0.327

(34, 37) v 0.003


(0, 4) f 5 6 56 55

(0, 5) f 6 7 57 56

(0, 6) f 7 8 58 57

(0, 7) f 8 9 59 58

(0, 8) f 9 10 60 59

(0, 9) f 10 11 61 60

(0, 10) f 11 12 62 61

(0, 11) f 12 13 63 62

(0, 12) f 13 14 64 63

(0, 13) f 14 15 65 64

(0, 14) f 15 16 66 65

(0, 15) f 16 17 67 66

(0, 16) f 17 18 68 67

(0, 17) f 18 19 69 68

(0, 18) f 19 20 70 69

(0, 19) f 20 21 71 70

(0, 20) f 21 22 72 71

(0, 21) f 22 23 73 72

(0, 22) f 23 24 74 73

(0, 23) f 24 25 75 74

(0, 24) f 25 26 76 75

(0, 25) f 26 27 77 76

(0, 26) f 27 28 78 77

(0, 27) f 28 29 79 78

(0, 28) f 29 30 80 79

(0, 29) f 30 31 81 80

(0, 30) f 31 32 82 81

(0, 31) f 32 33 83 82

(0, 32) f 33 34 84 83

(0, 33) f 34 35 85 84

(0, 34) f 35 36 86 85

(0, 35) f 36 37 87 86

(0, 36) f 37 38 88 87

(0, 37) f 38 39 89 88

(0, 38) f 39 40 90 89

(0, 39) f 40 41 91 90

(0, 40) f 41 42 92 91

(0, 41) f 42 43 93 92

(0, 42) f 43 44 94 93

(0, 43) f 44 45 95 94

(0, 44) f 45 46 96 95

(0, 45) f 46 47 97 96

(0, 46) f 47 48 98 97

(0, 47) f 48 49 99 98

(0

(20, 16) f 1017 1018 1068 1067

(20, 17) f 1018 1019 1069 1068

(20, 18) f 1019 1020 1070 1069

(20, 19) f 1020 1021 1071 1070

(20, 20) f 1021 1022 1072 1071

(20, 21) f 1022 1023 1073 1072

(20, 22) f 1023 1024 1074 1073

(20, 23) f 1024 1025 1075 1074

(20, 24) f 1025 1026 1076 1075

(20, 25) f 1026 1027 1077 1076

(20, 26) f 1027 1028 1078 1077

(20, 27) f 1028 1029 1079 1078

(20, 28) f 1029 1030 1080 1079

(20, 29) f 1030 1031 1081 1080

(20, 30) f 1031 1032 1082 1081

(20, 31) f 1032 1033 1083 1082

(20, 32) f 1033 1034 1084 1083

(20, 33) f 1034 1035 1085 1084

(20, 34) f 1035 1036 1086 1085

(20, 35) f 1036 1037 1087 1086

(20, 36) f 1037 1038 1088 1087

(20, 37) f 1038 1039 1089 1088

(20, 38) f 1039 1040 1090 1089

(20, 39) f 1040 1041 1091 1090

(20, 40) f 1041 1042 1092 1091

(20, 41) f 1042 1043 1093 1092

(20, 42) f 1043 1044 1094 1093

(20, 43) f 1044 1045 1095 1094

(20, 44) f 1045 1046 1096 1095

(20, 45) f 1046 1047 1097 1096

(20, 46) f 1047 1048 1098 1097

(20, 47)

(35, 38) f 1789 1790 1840 1839

(35, 39) f 1790 1791 1841 1840

(35, 40) f 1791 1792 1842 1841

(35, 41) f 1792 1793 1843 1842

(35, 42) f 1793 1794 1844 1843

(35, 43) f 1794 1795 1845 1844

(35, 44) f 1795 1796 1846 1845

(35, 45) f 1796 1797 1847 1846

(35, 46) f 1797 1798 1848 1847

(35, 47) f 1798 1799 1849 1848

(35, 48) f 1799 1800 1850 1849

(36, 0) f 1801 1802 1852 1851

(36, 1) f 1802 1803 1853 1852

(36, 2) f 1803 1804 1854 1853

(36, 3) f 1804 1805 1855 1854

(36, 4) f 1805 1806 1856 1855

(36, 5) f 1806 1807 1857 1856

(36, 6) f 1807 1808 1858 1857

(36, 7) f 1808 1809 1859 1858

(36, 8) f 1809 1810 1860 1859

(36, 9) f 1810 1811 1861 1860

(36, 10) f 1811 1812 1862 1861

(36, 11) f 1812 1813 1863 1862

(36, 12) f 1813 1814 1864 1863

(36, 13) f 1814 1815 1865 1864

(36, 14) f 1815 1816 1866 1865

(36, 15) f 1816 1817 1867 1866

(36, 16) f 1817 1818 1868 1867

(36, 17) f 1818 1819 1869 1868

(36, 18) f 1819 1820 1870 1869

(36, 19) f 1820 1821 1871 1870

(36, 20) f 1821 18

In [73]:
x = x.flatten()
y = y.flatten()
z = z.flatten()

In [53]:
fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)])
fig.show()